![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook uses notebook 2 as starting point and adds a 2D convolution filter accelerator into the video pipeline. Three versions of the 2D filter accelerator are available:
* Implemented in software running on the A72 cores (PS) using standard OpenCV functions
* Implemented in the PL using the Vitis Vision libraries and high-level synthesis (HLS)
* Implemented in the AI Engine (AIE) using one AIE core with HLS-based data mover in the PL

The 2D filter has a fixed kernel size of 3x3. It operates on the luma channel of a YUY2 image; the chroma channel is looped back unmodified.For the AIE version of the 2D filter, the chroma loopback is performed inside the data mover and only the chroma channel is forwarded to the AIE core that performs the convolution.
The PL 2D filter performs the chroma loopback inside the kernel itself. In addition, the PL version has run-time programmable kernel coefficients and presets as well as well image dimensions. For the AIE version, the kernel coefficients are hard-coded to "horizontal sobel" and the image dimensions are fixed at 1280x720. This limitation will be fixed in a future release.

For more information on the 2D convolution filter operation, see here: https://en.wikipedia.org/wiki/Kernel_(image_processing).

The various 2D filter accelerators are integrated into Gstreamer using the Vitis Video Analytics SDK (VVAS) which provides a set of plugins and an abstraction layer on top of the Xilinx run-time (XRT). For more information, see here: https://xilinx.github.io/VVAS/.

The video pipeline is composed of the following GStreamer elements:
* The ``mediasrcbin`` element is used to capture video from a V4L2 device
* The ``ivas_xfilter`` element is used to wrap the 2D convolution filter accelerator library
* The ``jpegenc`` element is used to compress the raw video format to JPEG.
* The ``appsink`` element is used to make the JPEG frames available to the jupyter notebook where they are displayed.

In this notebook, you will:
1. Create a GStreamer video pipeline that captures video from a V4L2 device, performs the 2D convolution, and displays the processed video inside this notebook.
2. Switch between the different 2D filter implementations: PS, PL, or AIE
3. Modify the filter presets or coefficients for the PS/PL implementations
4. Create a GStreamer pipeline graph and view it inside this notebook.

**Note:** The same 2D filter element can be applied in similar fashion to other notebooks.

# 2. Imports and Initialization

Import all python modules required for this notebook. 

In [ ]:
from IPython.display import Image, display, clear_output
import glob
import subprocess
import pydot
import sys
import os
import gi
gi.require_version('Gst', '1.0')
gi.require_version("GstApp", "1.0")
from gi.repository import GObject, GLib, Gst, GstApp

This is the Base TRD notebook 6 (nb6).

In [ ]:
nb = "nb6"

Create a directory for saving the pipeline graph as dot file. Set the GStreamer debug dot directory environment variable to point to that directory.

In [ ]:
dotdir = "/home/root/gst-dot/" + nb
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

Validate the correct Vitis Overlay is available in the platform for this notebook.

In [ ]:
def xbutil_program_xclbin():
    xclbin = "/media/sd-mmcblk0p1/binary_container_1.xclbin"
    if os.path.exists(xclbin):
        subprocess.run(['xbutil', 'program', '-p', xclbin], check=True)

def xbutil_query_cu(cu):
    proc = subprocess.run(['xbutil', 'query'], capture_output=True, encoding='utf8')
    for line in proc.stdout.splitlines():
        if cu in line:
            return
    raise Exception("Unable to find compute unit \'" + cu + "\'. Make sure the correct Vitis overlay is used.")

xbutil_program_xclbin()
xbutil_query_cu("filter2d_pl_accel")
#xbutil_query_cu("filter2d_aie_accel")

Initialize the GStreamer library. Enable debug by setting the debug string, set default to level 1 for all categories.

In [ ]:
Gst.init(None)
Gst.debug_set_threshold_from_string('*:1', True)

# 3. Create and Configure the GStreamer Elements

The ``get_media_by_device`` function returns the matching media node for a given video capture source. The following sources are supported in this notebook:
* ``vivid`` : virtual video device (default)
* ``usb`` : requires USB webcam
* ``mipi`` : platform1 only, requires FMC card
* ``hdmi`` : platform3 only, requires HDMI input

In [ ]:
def get_media_dev_by_name(src):
    sources = {
        'vivid' : 'vivid',
        "usb" : 'uvcvideo',
        'mipi' : 'vcap_csi',
        'hdmi' : 'vcap_hdmi'
    }
    devices = glob.glob('/dev/media*')
    for dev in devices:
        proc = subprocess.run(['media-ctl', '-d', dev, '-p'], capture_output=True, encoding='utf8')
        for line in proc.stdout.splitlines():
            if sources[src] in line:
                return dev

Select the ``source`` based on available media devices for this platform. The default source is set to ``vivid``. Update the value next to the comment to select USB webcam or MIPI single-sensor if connected.

In [ ]:
source = "usb" # Change source to vivid, usb, mipi, hdmi

media_device = get_media_dev_by_name(source) 
if media_device is None:
    raise Exception('Unable to find video source ' + source + '. Make sure the device is plugged in, powered, and the correct platform is used.')

The source pads of the ``mediasrcbin`` element are created dynamically when it detects the incoming stream. The ``pad-added`` signal is emitted and this ``pad_added`` callback function is executed. It links the source pads of the mediasrcbin elements to the sink pads of the ``caps`` elements.

Set the ``io-mode`` on the pad which propagates to the ``v4l2src`` node. If MIPI is selected, set the I/O mode to DMABUF (https://www.kernel.org/doc/html/v4.16/driver-api/dma-buf.html) which allows sharing of video buffers in 0-copy fashion between the source and sink elements. Otherwise, set the I/O mode to mmap.

In [ ]:
def pad_added(element, pad):
    sink_pad = caps.get_static_pad("sink")
    if not sink_pad.is_linked():
        pad.link(sink_pad)
        if source == "mipi" or source == "hdmi":
            pad.set_property("io-mode", "dmabuf")
        else:
            pad.set_property("io-mode", "mmap")

Create the ``mediasrcbin`` element which is a bin element that uses the standard ``v4l2src`` element inside. Set the following some properties:
* Set the ``media-device`` property to the desired media device node
* Register the above ``pad_added`` callback function with the ``pad-added`` signal of the ``mediasrcbin`` element.

In [ ]:
src = Gst.ElementFactory.make("mediasrcbin")
src.set_property("media-device", media_device)
src.connect("pad_added", pad_added);

Create a caps filter element to set the desired resolution (width and height) and format. The caps filter is configured to parse the mentioned properties from a string.

The default resolution is set to 1280x720 and the format to YUY2 as those are commonly supported by USB webcams.

In [ ]:
width = 1280
height = 720
fmt = "YUY2"

cap_string = "video/x-raw, width=" + str(width) + ", height=" + str(height) + ", format=" + fmt
if source == "mipi" or source == "hdmi":
    fps = "60/1"
    cap_string = cap_string + ", framerate=" + fps

caps = Gst.ElementFactory.make("capsfilter")
cap = Gst.Caps.from_string(cap_string)
caps.set_property("caps", cap)

Create the ``ivas_xfilter`` element. The ``kernels-config`` property is set to a json file path which contains configuration paramaters of the 2D filter implementation to be loaded at initialization time. Each implementation (PL, AIE, SW) has a dedicated json file. The ``dynamic-config`` property accepts a json string that is passed to the kernel each time the start function is called. This allows kernel parameters to be updated dynamically while the pipeline is running.

The json file captures the following filter2d specific configuration paramaters:
* ``debug_level`` : level for debug messages of this kernel library
* ``in_fourcc`` : input format in fourcc notation; this TRD support 'YUY2' only
* ``out_fourcc`` : output format in fourcc notation; this TRD supports 'YUY2' only
* ``filter_preset`` : filter preset name that translates to a set of coefficients
* ``filter_coefficients`` : 3x3 matrix of kernel coefficients; valid only if ``filter_preset`` is set to 'custom'

**Note:** The AIE filter2d currently only supports 1280x720. If any other resolution is used, an assertion error is thrown. Please use the ``filter2d_pl_accel`` kernel instead or change the resolution accordingly. This limitation will be fixed in a future release.

In [ ]:
jsondir = "/usr/share/ivas/base-trd/"

filter_kernels = ["PL", "AIE", "SW"]
filter_kernel = filter_kernels[0] # Change filter kernel to PL, AIE or SW via list index
print("Selected filter2d kernel: " + filter_kernel)

if filter_kernel == "PL":
    jfile = jsondir + "kernel_xfilter2d_pl.json"
elif filter_kernel == "AIE":
    assert width == 1280 and height == 720, "The AIE filter2d kernel only supports 720p!"
    # TODO
else: # filter_kernel == "SW"
    jfile = jsondir + "kernel_xfilter2d_sw.json"

filter2d = Gst.ElementFactory.make("ivas_xfilter")
filter2d.set_property("kernels-config", jfile)

The PL filter2d has two ways of programming the filter coefficients:
1. by setting the ``filter_preset`` which translates to a set of coefficients
2. by setting the ``filter_coefficients`` directly

First, we look at how to set the ``filter_preset`` parameter. The below command returns a list of supported presets to choose from.

In [ ]:
plist = [
    "blur",
    "edge",
    "horizontal edge",
    "vertical edge",
    "emboss",
    "horizontal gradient",
    "vertical gradient",
    "identity",
    "sharpen",
    "horizontal sobel",
    "vertical sobel",
    "custom"
]

def print_presets():
    print("Supported filter presets:\n")
    print('\n'.join(plist) + '\n')
    
print_presets()

Set the ``filter_preset`` parameter to one of the supported values listed above e.g. "horizontal sobel" by passing the json string via the ``dynamic-config`` element property. The kernel library reads the preset and programs the filter coefficients behind the scene.

**Note:** The filter-preset for the AIE filter2d is hard-coded to "horizontal sobel". Changing the property value will not affect the output image. This limitation will be fixed in a future release.

In [ ]:
def set_preset(val):
    if val in plist:
        jstring = '{ "filter_preset" : "' +  val + '" }'
        #print(jstring)
        filter2d.set_property("dynamic-config", jstring)
    else:
        raise Exception("Unsupported filter preset \'" + val + "\'")

set_preset("horizontal sobel")

The second way is to explicitly program the coefficients via the ``filter_coefficients`` parameter. Uncomment the last line in the next cell to program the coefficients via the ``dynamic-config`` element property.

The filter coefficients are a 3x3 matrix of ``short int`` values. The default values in the below code snippet correspond to the identity matrix which results is a simple passthrough. The identity coefficients are as follows:
```
 0  0  0
 0  1  0
 0  0  0
```

To match the coefficients for "horizontal sobel", use the following matrix:
```
 1  2  1
 0  0  0
-1 -2 -1
```

**Note:** The ``coefficients`` for the AIE filter2d are hard-coded to "horizontal sobel". Changing the coefficients values will not affect the output image. This limitation will be fixed in a future release.

In [ ]:
def set_coeff(val):
    jstring = '{ "filter_coefficients" : ' + val + ' }'
    #print(jstring)
    filter2d.set_property("dynamic-config", jstring)

#set_coeff("[[0, 0, 0], [0, 1, 0], [0, 0, 0]]")

Create the ``jpegenc`` element to compress the YUY2 video frame to JPEG.

In [ ]:
jpegenc = Gst.ElementFactory.make("jpegenc")  

Create a callback function ``new_sample`` that retrieves the JPEG data from a GStreamer buffer object and passes it to the ``display`` function of the ``IPython.display`` module which displays the video frame inside the notebook.

In [ ]:
def new_sample(sink):   
    sample = sink.pull_sample()
    buffer = sample.get_buffer()
    ret, info = buffer.map(Gst.MapFlags.READ)
    
    display(Image(data=info.data))
    clear_output(wait=True)
    
    buffer.unmap(info)
    
    return Gst.FlowReturn.OK

Create the ``appsink`` element and set some properties:
* Set the ``drop`` property to ``True`` to drop old buffers when the buffer queue is full
* Set the ``max-buffers`` property to 0 to queue an unlimited number of buffers
* Set the ``emit-signals`` property to ``True`` to emit the ``new-sample`` signal

Register the above ``new_sample`` callback function with the ``new-sample`` signal of the ``appsink`` element.

In [ ]:
sink = Gst.ElementFactory.make("appsink")
sink.set_property("drop", True)
sink.set_property("max_buffers", 0)
sink.set_property("emit-signals", True)
sink.connect("new-sample", new_sample);

# Uncomment the below code to read back the newly set property values
#print("appsink properties: ")
#print("drop: " + str(sink.get_property("drop")))
#print("max_buffers: " + str(sink.get_property("max_buffers")))
#print("emit-signals: " + str(sink.get_property("emit-signals")))

# 4. Create and Run the GStreamer Pipeline

Create the pipeline, add all elements, and link them together.

**Note:** Newly added or modified lines for adding in the filter2d element are marked with code comments

In [ ]:
pipeline = Gst.Pipeline.new(nb)

pipeline.add(src)
pipeline.add(caps)
pipeline.add(filter2d) # Create the filter2d element
pipeline.add(jpegenc)
pipeline.add(sink)

caps.link(filter2d) # Link the fiter2d element's sink pad to the caps element 
filter2d.link(jpegenc) # Link the filter2d element's source pad to the jpegenc element
jpegenc.link(sink);

The ``bus_call`` function listens on the bus for ``EOS`` and ``ERROR`` events. If any of these events occur, stop the pipeline (set to ``NULL`` state) and quit the main loop.

In case of an ``ERROR`` event, parse and print the error message.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

Start the pipeline (set to ``PLAYING`` state), create the main loop and listen to messages on the bus. Register the ``bus_call`` callback function with the ``message`` signal of the bus. Start the main loop.

The video frames will be displayed below the following code cell. 

To stop the pipeline, click the square shaped icon labelled 'Interrupt the kernel' in the top menu bar. Create a dot graph of the pipeline topology before stopping the pipeline. Quit the main loop.

In [ ]:
pipeline.set_state(Gst.State.PLAYING);

loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    pass

# 5. View the GStreamer Pipeline Graph

Convert the dot file to png and display the pipeline graph. The image will be displayed below the following code cell. Double click on the generate image file to zoom in.

**Note:** This step may take a few seconds.

In [ ]:
dotfile = dotdir + "/" + nb + ".dot"
graph = pydot.graph_from_dot_file(dotfile, 'utf-8')
display(Image(graph[0].create(None, 'png', 'utf-8')))

# 6. Summary

In this notebook you learned how to:
1. Create a GStreamer pipeline that demonstrates how to capture video from a V4L2 device, process the video using a 2D convolution filter, and play it back inside the jupyter notebook
2. Configure the 2D filter for different implementation modes: PS, PL, or AIE
3. Program the filter coefficients directly or via presets
4. Export the pipeline topology as a dot file image and display it in the notebook

<center>Copyright© 2019 Xilinx</center>